In [1]:
path = !(echo $(pwd))

In [2]:
import os
import sys
sys.path.insert(0, os.path.dirname(os.path.dirname(path[0])))

import numpy as np
import chess
import tensorflow
from tensorflow import keras
import keras_nlp

from src.preprocessing.board_representation import token_sequence_to_board
from src.evaluation.visualisation import *
from src.utils.data_loader import load_train_test
from src.modeling.custom_losses import sum_squared_loss, pc_column_reg, pc_plane_reg, num_pc_reg, custom_regularized_loss
from src.modeling.model import PositionPredictionHead

2023-10-15 15:33:45.540032: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-10-15 15:33:45.632203: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-10-15 15:33:45.632217: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-10-15 15:33:46.193613: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-

Using TensorFlow backend


In [3]:
print(tensorflow.__version__)

2.11.1


In [4]:
train_data, test_data = load_train_test("../../data", "position_tensor_000")
print(len(train_data), len(test_data))

90000 10000


In [5]:
backend = keras.models.load_model("../../model/20231015152500_autoencoder.tf")

2023-10-15 15:33:49.795665: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2023-10-15 15:33:49.795687: W tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: UNKNOWN ERROR (303)
2023-10-15 15:33:49.795703: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (pafrank-hystou): /proc/driver/nvidia/version does not exist
2023-10-15 15:33:49.795982: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [6]:
model = PositionPredictionHead(backend=backend)

# Encoder and decoder evaluation

In [7]:
index = 3141

In [8]:
sample = test_data[index].reshape((1,69))

In [9]:
reconstruction = model.predict_on_batch(sample)

In [ ]:
tensorflow.math.count_nonzero(tensorflow.subtract(sample, reconstruction), axis=-1)

In [12]:
backend.evaluate(sample, reconstruction, return_dict=True), model.evaluate(x=sample)

1/1 [==============================] - 0s 31ms/step - loss: 4.8698e-04 - sparse_categorical_accuracy: 1.0000


({'loss': 0.00048697966849431396, 'sparse_categorical_accuracy': 1.0},
 <tf.Tensor: shape=(1,), dtype=int64, numpy=array([0])>)

In [15]:
sample_board = token_sequence_to_board(sample[0])
reconstructed_board = token_sequence_to_board(reconstruction[0].numpy())

In [16]:
print_board(sample_board)

r . . . . r k .
p p . . . p p p
. . p . . . b .
b . . N P . . .
. . . . n P P q
. B . . B Q . P
P P . . . . . .
R . . . . R K .




In [17]:
print_board(reconstructed_board)

r . . . . r k .
p p . . . p p p
. . p . . . b .
b . . N P . . .
. . . . n P P q
. B . . B Q . P
P P . . . . . .
R . . . . R K .




# Position similarity search

In [ ]:
chess_positions = {
    "game_1_catalan_opening" : chess.Board(fen="r1bqr1k1/pp1nnpbp/2p3p1/4p3/4P3/4BNP1/PPPQ1PBP/RN2R1K1 w - - 2 11"), # game 1 - catalan opening
    "game_1_one_move_later" : chess.Board(fen="r1bqr1k1/pp1nnpbp/2p3pB/4p3/4P3/5NP1/PPPQ1PBP/RN2R1K1 b - - 3 11"),
    "game_1_few_moves_later" : chess.Board(fen="r3r3/ppq1npkp/2p3p1/2n1p2P/4P1b1/5NP1/PPPQ1PB1/RN2R1K1 w - - 1 15"),
    "game_1_middle_game" : chess.Board(fen="7r/p3npk1/2p4r/1p2N1p1/1P2P3/2P3P1/P4PK1/3RR3 w - - 0 28"),
    "game_1_end_game" : chess.Board(fen="8/8/8/1N3k2/1n3Pp1/6P1/5K2/8 w - - 0 43"),
    "game_2_catalan_opening" : chess.Board(fen="rn1qk2r/pp2bppp/2p5/3p1b2/3PnB2/2N2NP1/PP2PPBP/R2Q1RK1 b kq - 1 9"),
    "game_2_middle_game" : chess.Board(fen="r2r2k1/pQ3ppp/3q4/3P1b2/2R5/6P1/P4PBP/5RK1 b - - 0 21")
}